# Notebook for deploying and evaluate model on aws

On this notebook we will be training and evaluating a dog breed classifier based on feature extraction of a resnet50 archtecture!

---

In [1]:
# import aws sagemaker apis
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

In [3]:
# specify where to upload in S3
data_dir = './data/dataset/'
prefix = 'dog-breed-data'

s3_keys = [obj.key for obj in boto3.resource('s3').Bucket(bucket).objects.all()]
# upload to S3 - Due to the size of the dataset it takes some time to finish...
if any(prefix in s for s in s3_keys):
    input_data = f"s3://{bucket}/{prefix}"
else:
    input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-031843945636/dog-breed-data


## PyTorch Estimator

Here we define the pytorch estimator for the problem. It's this api that will handle all the load of creating the training job and deploy.

It's needed to specify the train script on it though. the pytorch framework version, train_instance_type and model hyperparameters are really important atributes of it. 

In [4]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source', # this should be just "source" for your code
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'output_dim': 120,
                        'hidden_dim': 1000,
                        'epochs': 20, # could change to higher
                        'batch-size': 32,
                        'lr': 0.01
                    })

Here we start the training job!

In [5]:
estimator.fit({'train': os.path.join(input_data, 'train'), 'test': os.path.join(input_data, 'test')})

2020-05-22 01:41:23 Starting - Starting the training job...
2020-05-22 01:41:25 Starting - Launching requested ML instances.........
2020-05-22 01:43:06 Starting - Preparing the instances for training.........
2020-05-22 01:44:24 Downloading - Downloading input data..................
2020-05-22 01:47:33 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-22 01:48:01,182 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-22 01:48:01,209 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-22 01:48:04,363 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-22 01:48:04,723 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-05-22 01:48:04,724 sagemaker-containers INFO     Generating setup.cfg
2020-0

### Deploying the model for evaluation

In [6]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------!

In [12]:
import numpy as np
from pprint import pprint
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [8]:
from source.utils import get_validation_dataset

dataset = get_validation_dataset('data/dataset/val/')

In [9]:
actual_label = dataset.targets
predicted_labels = [np.argmax(predictor.predict(data.unsqueeze(0)), 1)[0] for data, label in dataset] # takes some time

In [10]:
metrics = {key: precision_recall_fscore_support(actual_label, predicted_labels, average=key) for key in ["micro", "macro", "weighted"]}

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
print(f"Model accuracy: {accuracy_score(actual_label, predicted_labels):.3f}")
print(f'Model metrics: \n'
      f" (micro)   precision - {metrics['micro'][0]:.3f}, recall - {metrics['micro'][1]:.3f}, f1score - {metrics['micro'][2]:.3f}\n"
      f" (macro)   precision - {metrics['macro'][0]:.3f}, recall - {metrics['macro'][1]:.3f}, f1score - {metrics['macro'][2]:.3f}\n"
      f"(weighted) precision - {metrics['weighted'][0]:.3f}, recall - {metrics['weighted'][1]:.3f}, f1score - {metrics['weighted'][2]:.3f}")

Model accuracy: 0.777
Model metrics: 
 (micro)   precision - 0.777, recall - 0.777, f1score - 0.777
 (macro)   precision - 0.808, recall - 0.777, f1score - 0.764
(weighted) precision - 0.826, recall - 0.777, f1score - 0.775


## Deleting the endpoint

In [14]:
predictor.delete_endpoint()